# Graphing module

## Mapping and syntax processing (AST)

### Syntax processing (AST)
- Here there are 3 main options available (that I currently know of)
    - ANTLR (more community support)
    - tree-sitter (speed, lazy-evaluation)
    - Bison (more for config files)

Because currently we have the most done on tree-sitter, the focus will be on that, but it's always possible to switch or write a custom AST Parser since our usecase is quite unique

In [6]:
from dataclasses import dataclass
import sys
from enum import unique, Enum
from unicodedata import name

# This is used to be able to import from src directory
sys.path.append("../")


from src.repository_processing import files_from_repository
from tree_sitter_languages import get_language, get_parser
from tree_sitter import Language, Parser, Node
from abc import ABC, abstractmethod, abstractproperty


# TODO: Add Result monad to properties to handle errors
# @property is not cached, if more performance needed use @functools.cached_property
@dataclass
class LanguageInfo(ABC):
    """abstract base class (impl) for holding language information and parsers for AST"""

    @property
    @abstractmethod
    def name() -> str:
        ...

    @property
    @abstractmethod
    def extensions() -> tuple[str]:
        ...

    @property
    def language(self) -> Language:
        return get_language(self.name)

    @property
    def parser(self) -> Parser:
        return get_parser(self.name)


class Python(LanguageInfo):
    name: str = "python"
    extensions: tuple[str] = ".py"


class JavaScript(LanguageInfo):
    name: str = "javascript"
    extensions: tuple[str] = ".js"


# matching the file name to the correct language class using it's extension list
def get_language_class(file_path: str) -> LanguageInfo:
    for language_class in LanguageInfo.__subclasses__():
        for extension in language_class.extensions:
            if file_path.endswith(extension):
                return language_class
    raise ValueError("No language class found for file path")


python_language = Python()

import networkx as nx
from pyvis.network import Network


def print_tree_structure(node: Node, graph: nx.Graph):
    # Print the node's value
    print(node.id, node.type, node.text)

    # Add the node to the graph
    graph.add_node(node.id, title=str(node.text))

    # Recursively print the structure of the node's children
    for child in node.children:
        print_tree_structure(child, graph)
        graph.add_edge(node.id, child.id)


def process_dependancy_logic(node: Node, graph: nx.Graph):
    # Add the node to the graph
    graph.add_node(node.id, title=str(node.text))

    # Check if the node is of a specific type that indicates dependancy
    if node.type in ["import_declaration", "function_call", "variable_reference"]:
        # Add an edge to the dependancy node in the graph
        dependancy_node = node.children[0]
        graph.add_edge(node.id, dependancy_node.id)

    # Check if the node is of a specific type that indicates logic
    elif node.type in ["if_statement", "for_statement", "while_statement"]:
        # Add an edge to the logic node in the graph
        logic_node = node.children[0]
        graph.add_edge(node.id, logic_node.id)

    # Recursively process the structure of the node's children
    for child in node.children:
        process_dependancy_logic(child, graph)


files = files_from_repository("https://github.com/Foxicution/repo-review")
for file in files:
    if file.path.endswith(python_language.extensions):
        print(file.path)
        # Parse the file and get the root node of the tree
        tree = python_language.parser.parse(file.content)
        root_node = tree.root_node

        # Create an empty graph
        graph = nx.Graph()

        # Walk through the tree and print its structure
        # print_tree_structure(root_node, graph)
        process_dependancy_logic(root_node, graph)

        # Draw the graph
        nt = Network("1000px", "2000px")
        nt.options.physics.enabled = False
        nt.show_buttons()
        # populates the nodes and edges data structures
        nt.from_nx(graph)
        nt.show("nx.html")
        break


/main.py
